## requirements

In [ ]:
conda env remove --name ev4nlpCr
conda create -n ev4nlpCr2 python=3.10.13
conda activate ev4nlpCr2
conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia
pip install numpy tqdm pandas matplotlib packaging argparse transformers


conda remove --name myenv --all

## GET TRANSLATION5

### gpt-3.5-turbo API(1s/t)

In [1]:
# test openai.ChatCompletion.create
import openai
import time
import os
import re

with open('dataset/zh.source', 'r', encoding='utf-8') as file:
    ls_zhData = file.readlines()

with open('openai.api_key', 'r', encoding='utf-8') as file:
    openai.api_key = file.readlines()[0]

TPs = [
'Translate these sentences from Chinese to English:\n',  
'Answer with no quotes. What do these sentences mean in English?\n', 
'Please provide the English translation for these sentences:\n'
]


start_tpi = -1
start_nlsi = -1
# 获取当前文件夹中的所有文件
files = os.listdir('.')
# 筛选出所有满足条件的文件，获取其中的数字部分，并转化为整数
nums = [int(re.search(r'tp(\d+)_translated_sentences\.txt', f).group(1)) for f in files if re.match(r'tp(\d+)_translated_sentences\.txt', f)]
# 获取最大的数字，如果没有满足条件的文件则返回None
start_tpi = max(nums) if nums else None
# 如果存在满足条件的文件
if start_tpi is not None:
    # 读取对应文件的倒数第二行
    with open(f'tp{start_tpi}_translated_sentences.txt', 'r', encoding='utf-8') as f:
        lines = f.readlines()
        if len(lines) >= 2:
            s = re.search(r'\[(.*?)\]', lines[-1])
            s = s.group(1) if s else None
            start_nlsi = int(s)+1

tpi = 1
while tpi <= len(TPs):
    if start_tpi != -1:
        tpi = start_tpi
        start_tpi = -1
    tp = TPs[tpi - 1]  # adjust index here
    print('\nTP'+str(tpi)+':'+tp)
    # if start_tpi == -1:
    #     with open('tp'+str(tpi)+'_translated_sentences.txt', 'a+', encoding='utf-8') as f:
    #         f.write("TP%d: %s\n" % (tpi, tp))
    # 4. 循环询问GPT
    lsAns = []  # 用于存储GPT的回答
    nlsi = 1
    while nlsi <= len(ls_zhData):
        if start_nlsi != -1:
            nlsi = start_nlsi
            start_nlsi = -1
        unit = ls_zhData[nlsi - 1]  # adjust index here
        askContent = tp + "".join(unit)
        # print(askContent)
        # if True:
        try:
            completion = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                { 
                "role": "user",
                "content": askContent
                } 
        ]
        )
            ans = completion.choices[0].message["content"]
            lsAns.append(ans)
            print("%d/%d" % (nlsi, len(ls_zhData)), end=' |')
            if nlsi % 10 == 0:
                print()
        except Exception as e:
            ans = "Error[%d,%d]: %s" % (tpi, nlsi, str(e))
            print(ans)
            start_tpi = tpi
            start_nlsi = nlsi + 1
        # print(ans)
        # print('\n'+'-'*10+'\n')
    
        # 5. 保存C到文件
        with open('tp'+str(tpi)+'_translated_sentences.txt', 'a+', encoding='utf-8') as f:
            f.write("[%d]:%s\n" % (nlsi, ans))
        # time.sleep(0.02)
        nlsi += 1
    tpi += 1


TP3:Please provide the English translation for these sentences:

849/1200 |850/1200 |
851/1200 |852/1200 |853/1200 |854/1200 |855/1200 |856/1200 |Error[3,857]: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)
858/1200 |859/1200 |860/1200 |
861/1200 |862/1200 |863/1200 |864/1200 |865/1200 |866/1200 |867/1200 |868/1200 |869/1200 |870/1200 |
871/1200 |872/1200 |873/1200 |874/1200 |875/1200 |876/1200 |877/1200 |878/1200 |879/1200 |880/1200 |
881/1200 |882/1200 |883/1200 |884/1200 |885/1200 |886/1200 |887/1200 |888/1200 |889/1200 |890/1200 |
891/1200 |892/1200 |893/1200 |894/1200 |895/1200 |896/1200 |897/1200 |898/1200 |899/1200 |900/1200 |
901/1200 |902/1200 |903/1200 |904/1200 |905/1200 |906/1200 |907/1200 |908/1200 |909/1200 |910/1200 |
911/1200 |912/1200 |913/1200 |914/1200 |915/1200 |916/1200 |917/1200 |918/1200 |919/1200 |920/1200 |
921/1200 |922/1200 |923/1200 |924/1200 |925/1200 |926/1200 |927/1200 |928/1200 |929/1200 |930/

#### Individual Translation for Fix

In [2]:
# test openai.ChatCompletion.create

import openai
import time
import os
import re

with open('dataset/zh.source', 'r', encoding='utf-8') as file:
    ls_zhData = file.readlines()


with open('openai.api_key', 'r', encoding='utf-8') as file:
    openai.api_key = file.readlines()[0]

TPs = [
'Translate these sentences from Chinese to English:\n',  
'Answer with no quotes. What do these sentences mean in English?\n', 
'Please provide the English translation for these sentences:\n'
]


tpi = 3
nlsi = 1001
if True:
    tp = TPs[tpi - 1]  # adjust index here
    print('\nTP'+str(tpi)+':'+tp)
    lsAns = []  # 用于存储GPT的回答
    if True:
        unit = ls_zhData[nlsi - 1]  # adjust index here
        askContent = tp + "".join(unit)
        print(askContent)
        try:
            completion = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                { 
                "role": "user",
                "content": askContent
                } 
        ]
        )
            ans = completion.choices[0].message["content"]
            lsAns.append(ans)
            print("%d/%d" % (nlsi, len(ls_zhData)), end=' |')
        except Exception as e:
            ans = "Error[%d,%d]: %s" % (tpi, nlsi, str(e))
            print(ans)
            start_tpi = tpi
            start_nlsi = nlsi + 1
        print(ans)


TP3:Please provide the English translation for these sentences:

Please provide the English translation for these sentences:
这句话是引用古书里的。

1001/1200 |This sentence is a quotation from an ancient book.


### NLLB API(1s/t)

#### NLLB Sample
https://huggingface.co/docs/transformers/model_doc/nllb

In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# 初始化分词器和模型，设置源语言为中文（'zh'），目标语言为英文（'eng_Latn'）
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M", src_lang="zh")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

# 待翻译的中文句子
chinese_sentence = "请问你来自哪里？"

# 对句子进行编码
inputs = tokenizer(chinese_sentence, return_tensors="pt")

# 使用模型生成翻译的输出，并强制目标语言为英文
translated_tokens = model.generate(
    **inputs, forced_bos_token_id=tokenizer.lang_code_to_id["eng_Latn"]
)

# 解码模型输出的tokens，得到翻译后的英文句子
translated_sentence = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

print(translated_sentence)  # 输出翻译后的英文句子

Please ask me where you come from.


#### NLLB translation
- run code with stop
- clear 'zh' in tanslated output
- run second cell

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from tqdm import tqdm

# 检查是否有可用的 CUDA 设备并设置 device 变量
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 初始化 tokenizer 和 model，并将模型移动到设定的设备上
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M", src_lang="zh")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M").to(device)

def nllb_zh2en(srczh):
    chinese_sentence = srczh
    # 将输入数据移动到同一设备上
    inputs = tokenizer(chinese_sentence, return_tensors="pt").to(device)
    translated_tokens = model.generate(
        **inputs, forced_bos_token_id=tokenizer.lang_code_to_id["eng_Latn"]
    )
    translated_sentence = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    opten = translated_sentence.strip()
    return opten

list_path_src_dataset = ['zh.source1.CL_SA450', 'zh.source2.CT-SA350', 'zh.source3.LA400']

for path in list_path_src_dataset:
    path_src_dataset = 'dataset/' + path
    path_opt_tslt = 'translated/NLLB_API/' + 'NLLB_' + path + '_tslt_en.txt'
    
    with open(path_src_dataset, 'r', encoding='utf-8') as fin:
        list_src_dataset = fin.readlines()
    
    list_opt_tslt = []
    
    for line_src in tqdm(list_src_dataset, desc=f"Translating {path}"):
        list_opt_tslt.append(nllb_zh2en(line_src))
    
    with open(path_opt_tslt, 'a+', encoding='utf-8') as fout:
        for line_opt_tslt in list_opt_tslt:
            fout.write(line_opt_tslt + '\n')

In [16]:
for path in list_path_src_dataset:
    path_src_dataset = 'translated/NLLB_API/' + 'NLLB_' + path + '_tslt_en.txt'
    path_opt_tslt = 'translated/NLLB_API/' + 'NLLB_' + 'zh.source' + '_tslt_en.txt'
    with open(path_src_dataset, 'r', encoding='utf-8') as fin:
        with open(path_opt_tslt, 'a+', encoding='utf-8') as fout:
            for line_fin in fin:
                fout.write(line_fin)

### Google API(1s/t)
- using google translation directly
- combine translation text using cell below

In [18]:
for path in list_path_src_dataset:
    path_src_dataset = 'translated/Google_API/' + 'Google_' + path + '_tslt_en.txt'
    path_opt_tslt = 'translated/Google_API/' + 'Google_' + 'zh.source' + '_tslt_en.txt'
    with open(path_src_dataset, 'r', encoding='utf-8') as fin:
        with open(path_opt_tslt, 'a+', encoding='utf-8') as fout:
            for line_fin in fin:
                fout.write(line_fin)

### DeepL API(1s/t)
- using google translation directly
- combine translation text using cell below

In [19]:
for path in list_path_src_dataset:
    path_src_dataset = 'translated/DeepL_API/' + 'DeepL_' + path + '_tslt_en.txt'
    path_opt_tslt = 'translated/DeepL_API/' + 'DeepL_' + 'zh.source' + '_tslt_en.txt'
    with open(path_src_dataset, 'r', encoding='utf-8') as fin:
        with open(path_opt_tslt, 'a+', encoding='utf-8') as fout:
            for line_fin in fin:
                fout.write(line_fin)

## GET EVALUATION

### get ref+tslts

In [ ]:
# 读取文件
def read_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    return [line.strip() for line in lines]

ref_file = 'dataset/test.true.en'
tslt_file = 'translated/tp1_tslt_en.txt'

refs = read_file(ref_file)
tslts = read_file(tslt_file)

### BLEU

#### nltk corpus_bleu sample

In [22]:
# corpus_bleu SAMPLE
from nltk.tokenize import wordpunct_tokenize
from nltk.translate.bleu_score import corpus_bleu

# 注意：在references中，每个参考句子都是一个列表的元素，即使只有一个参考句子
references = [
                [['this', 'is', 'test', 'one'], ['this', 'is', 'a', 'test', 'one']],
                [['this', 'is', 'test', 'two'], ['this', 'is', 'a', 'test', 'two']]
             ]
candidates = [
                ['this', 'is', 'test', 'one'],
                ['this', 'is', 'test', 'two']
             ]
score = corpus_bleu(references, candidates)
print(score)

1.0


#### nltk sentence_bleu sample

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

# 注意：在references中，每个参考句子都是一个列表的元素，即使只有一个参考句子
references = [
    [['this', 'is', 'test', 'one'], ['this', 'is', 'a', 'test', 'one']],  # References for first sentence
    [['this', 'is', 'test', 'two'], ['this', 'is', 'a', 'test', 'two']]   # References for second sentence
]
candidates = [
    ['this', 'is', 'test', 'one'],  # Candidate for first sentence
    ['this', 'is', 'test', 'two']   # Candidate for second sentence
]

scores = []
for i in range(len(candidates)):
    # Compute BLEU score for each candidate against its corresponding references
    score = sentence_bleu(references[i], candidates[i])
    scores.append(score)
    print(f"BLEU score for sentence {i+1}: {score}")

average_score = sum(scores) / len(scores)
print(f"Average BLEU score: {average_score}")

#### BLEU evaluation

In [4]:
list_src_name = ['zh.source1.CL_SA450', 'zh.source2.CT-SA350', 'zh.source3.LA400', 'zh.source']
list_ref_name = ['test.true.en1.CL_SA450', 'test.true.en2.CT-SA350', 'test.true.en3.LA400', 'test.true.en']
# GPT individual
list_tslt_folder = ['NLLB', 'Google', 'DeepL']

from nltk.tokenize import wordpunct_tokenize
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import sentence_bleu
def read_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    return [line.strip() for line in lines]

for i_name in range(len(list_src_name)):
    src_name = list_src_name[i_name]
    ref_name = list_ref_name[i_name]

    path_ref = 'dataset/'+ref_name
    for tpi in range(1,3+1):
        id_src = ""
        if i_name != 3:
            id_src = str(i_name+1)
        path_tslt = 'translated/gpt-3.5-turbo_API/zh.source'+id_src+'_tp'+str(tpi)+'_tslt_en.txt'

        refs = read_file(path_ref)
        tslts = read_file(path_tslt)
        refs_tok = [[wordpunct_tokenize(sentence)] for sentence in refs]
        tslts_tok = [wordpunct_tokenize(sentence) for sentence in tslts]

        print("%s|gpt-3.5-turbo|TP%d|corpus_bleu = "%(src_name,tpi))
        print(corpus_bleu(refs_tok, tslts_tok))

    for tslt_name in list_tslt_folder:
        path_tslt = 'translated/'+tslt_name+'_API/'+tslt_name+'_'+src_name+'_tslt_en.txt'

        refs = read_file(path_ref)
        tslts = read_file(path_tslt)
        refs_tok = [[wordpunct_tokenize(sentence)] for sentence in refs]
        tslts_tok = [wordpunct_tokenize(sentence) for sentence in tslts]

        print("%s|%s|corpus_bleu = "%(src_name,tslt_name))
        print(corpus_bleu(refs_tok, tslts_tok))

zh.source1.CL_SA450|gpt-3.5-turbo|TP1|corpus_bleu = 
0.28158748958994156
zh.source1.CL_SA450|gpt-3.5-turbo|TP2|corpus_bleu = 
0.2676698411320771
zh.source1.CL_SA450|gpt-3.5-turbo|TP3|corpus_bleu = 
0.28026530722349025
zh.source1.CL_SA450|NLLB|corpus_bleu = 
0.20273545322393868
zh.source1.CL_SA450|Google|corpus_bleu = 
0.3321425941392817
zh.source1.CL_SA450|DeepL|corpus_bleu = 
0.2518351994659602
zh.source2.CT-SA350|gpt-3.5-turbo|TP1|corpus_bleu = 
0.3023981126241737
zh.source2.CT-SA350|gpt-3.5-turbo|TP2|corpus_bleu = 
0.30422142800103963
zh.source2.CT-SA350|gpt-3.5-turbo|TP3|corpus_bleu = 
0.3071332743320029
zh.source2.CT-SA350|NLLB|corpus_bleu = 
0.27158369215542183
zh.source2.CT-SA350|Google|corpus_bleu = 
0.38552247217268426
zh.source2.CT-SA350|DeepL|corpus_bleu = 
0.33097287199675074
zh.source3.LA400|gpt-3.5-turbo|TP1|corpus_bleu = 
0.33750745760901785
zh.source3.LA400|gpt-3.5-turbo|TP2|corpus_bleu = 
0.3140026825973613
zh.source3.LA400|gpt-3.5-turbo|TP3|corpus_bleu = 
0.3374409084

In [7]:
for i_name in range(len(list_src_name)):
    src_name = list_src_name[i_name]
    ref_name = list_ref_name[i_name]

    path_ref = 'dataset/'+ref_name
    for tpi in range(1,3+1):
        id_src = ""
        if i_name != 3:
            id_src = str(i_name+1)
        path_tslt = 'translated/gpt-3.5-turbo_API/zh.source'+id_src+'_tp'+str(tpi)+'_tslt_en.txt'

        refs = read_file(path_ref)
        tslts = read_file(path_tslt)
        refs_tok = [[wordpunct_tokenize(sentence)] for sentence in refs]
        tslts_tok = [wordpunct_tokenize(sentence) for sentence in tslts]

        bleu_scores_sentence_lv = []
        for i in range(len(refs_tok)):
            refi = refs_tok[i]
            tslti = tslts_tok[i]
            bleu_scores_sentence_lv.append(sentence_bleu(refi, tslti))
        avg_bleu_scores_sentence_lv = sum(bleu_scores_sentence_lv)/len(bleu_scores_sentence_lv)

        # print(bleu_scores_sentence_lv)

        print("%s|gpt-3.5-turbo|TP%d|sentence_bleu_avg = "%(src_name,tpi))
        print(avg_bleu_scores_sentence_lv)

    for tslt_name in list_tslt_folder:
        path_tslt = 'translated/'+tslt_name+'_API/'+tslt_name+'_'+src_name+'_tslt_en.txt'

        refs = read_file(path_ref)
        tslts = read_file(path_tslt)
        refs_tok = [[wordpunct_tokenize(sentence)] for sentence in refs]
        tslts_tok = [wordpunct_tokenize(sentence) for sentence in tslts]

        bleu_scores_sentence_lv = []
        for i in range(len(refs_tok)):
            refi = refs_tok[i]
            tslti = tslts_tok[i]
            bleu_scores_sentence_lv.append(sentence_bleu(refi, tslti))
        avg_bleu_scores_sentence_lv = sum(bleu_scores_sentence_lv)/len(bleu_scores_sentence_lv)

        print("%s|%s|sentence_bleu_avg = "%(src_name,tslt_name))
        print(avg_bleu_scores_sentence_lv)

zh.source1.CL_SA450|gpt-3.5-turbo|TP1|sentence_bleu_avg = 
0.18615906555104828
zh.source1.CL_SA450|gpt-3.5-turbo|TP2|sentence_bleu_avg = 
0.17181308259304906
zh.source1.CL_SA450|gpt-3.5-turbo|TP3|sentence_bleu_avg = 
0.18683988170872787
zh.source1.CL_SA450|NLLB|sentence_bleu_avg = 
0.10667113832002532
zh.source1.CL_SA450|Google|sentence_bleu_avg = 
0.23873082172811932
zh.source1.CL_SA450|DeepL|sentence_bleu_avg = 
0.14436123486561894
zh.source2.CT-SA350|gpt-3.5-turbo|TP1|sentence_bleu_avg = 
0.23984492293557577
zh.source2.CT-SA350|gpt-3.5-turbo|TP2|sentence_bleu_avg = 
0.2472777687780489
zh.source2.CT-SA350|gpt-3.5-turbo|TP3|sentence_bleu_avg = 
0.24719559619014989
zh.source2.CT-SA350|NLLB|sentence_bleu_avg = 
0.20906048467225247
zh.source2.CT-SA350|Google|sentence_bleu_avg = 
0.3339061660046424
zh.source2.CT-SA350|DeepL|sentence_bleu_avg = 
0.27705301172915076
zh.source3.LA400|gpt-3.5-turbo|TP1|sentence_bleu_avg = 
0.265747674271723
zh.source3.LA400|gpt-3.5-turbo|TP2|sentence_bleu_avg

## bert_score

### bert_score sample

In [10]:
# sentence-lv bert_score sample
from bert_score import score

cands = ['this is 1', 'this is 2']
refs = ['this is 1', 'this is 2']
P, R, F1 = score(cands, refs, lang='en')
print(P, R, F1)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([1.0000, 1.0000]) tensor([1.0000, 1.0000]) tensor([1.0000, 1.0000])


### bert_score evaluation

In [12]:
list_src_name = ['zh.source1.CL_SA450', 'zh.source2.CT-SA350', 'zh.source3.LA400', 'zh.source']
list_ref_name = ['test.true.en1.CL_SA450', 'test.true.en2.CT-SA350', 'test.true.en3.LA400', 'test.true.en']
# GPT individual
list_tslt_folder = ['NLLB', 'Google', 'DeepL']

import logging
from bert_score import score as bert_score
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def read_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    return [line.strip() for line in lines]

# Suppress warnings from transformers
logging.getLogger("transformers").setLevel(logging.ERROR)

for i_name in range(len(list_src_name)):
    src_name = list_src_name[i_name]
    ref_name = list_ref_name[i_name]

    path_ref = 'dataset/'+ref_name
    for tpi in range(1,3+1):
        id_src = ""
        if i_name != 3:
            id_src = str(i_name+1)
        path_tslt = 'translated/gpt-3.5-turbo_API/zh.source'+id_src+'_tp'+str(tpi)+'_tslt_en.txt'

        refs = read_file(path_ref)
        tslts = read_file(path_tslt)

        P, R, F1 = bert_score(tslts, refs, lang='en', device=device)
        avg_bert_score_P_R_F1=[P.mean().item(), R.mean().item(), F1.mean().item()]
        print("%s|gpt-3.5-turbo|TP%d|avg_bert_score[P, R, F1] = "%(src_name,tpi))
        print(avg_bert_score_P_R_F1)

    for tslt_name in list_tslt_folder:
        path_tslt = 'translated/'+tslt_name+'_API/'+tslt_name+'_'+src_name+'_tslt_en.txt'

        refs = read_file(path_ref)
        tslts = read_file(path_tslt)

        P, R, F1 = bert_score(tslts, refs, lang='en', device=device)
        avg_bert_score_P_R_F1=[P.mean().item(), R.mean().item(), F1.mean().item()]
        print("%s|%s|avg_bert_score[P, R, F1] = "%(src_name,tslt_name))
        print(avg_bert_score_P_R_F1)
        

zh.source1.CL_SA450|gpt-3.5-turbo|TP1|avg_bert_score[P, R, F1] = 
[0.9393912553787231, 0.9397478699684143, 0.9394848346710205]
zh.source1.CL_SA450|gpt-3.5-turbo|TP2|avg_bert_score[P, R, F1] = 
[0.9365745782852173, 0.9371306300163269, 0.9367662072181702]
zh.source1.CL_SA450|gpt-3.5-turbo|TP3|avg_bert_score[P, R, F1] = 
[0.9392860531806946, 0.9406151175498962, 0.9398643374443054]
zh.source1.CL_SA450|NLLB|avg_bert_score[P, R, F1] = 
[0.9275792241096497, 0.9262470602989197, 0.926801323890686]
zh.source1.CL_SA450|Google|avg_bert_score[P, R, F1] = 
[0.9474057555198669, 0.9438767433166504, 0.9455528259277344]
zh.source1.CL_SA450|DeepL|avg_bert_score[P, R, F1] = 
[0.9357784986495972, 0.9346756935119629, 0.9351478219032288]
zh.source2.CT-SA350|gpt-3.5-turbo|TP1|avg_bert_score[P, R, F1] = 
[0.9421859383583069, 0.9430044889450073, 0.9425522089004517]
zh.source2.CT-SA350|gpt-3.5-turbo|TP2|avg_bert_score[P, R, F1] = 
[0.9436308741569519, 0.9423050284385681, 0.9429275989532471]
zh.source2.CT-SA350|g

zh.source2.CT-SA350|NLLB|avg_bert_score[P, R, F1] = 
[0.9290854930877686, 0.9263474941253662, 0.9276657700538635]
zh.source2.CT-SA350|Google|avg_bert_score[P, R, F1] = 
[0.9500586986541748, 0.9480114579200745, 0.9489890933036804]
zh.source2.CT-SA350|DeepL|avg_bert_score[P, R, F1] = 
[0.9422419667243958, 0.9417366981506348, 0.9419476389884949]
zh.source3.LA400|gpt-3.5-turbo|TP1|avg_bert_score[P, R, F1] = 
[0.950654149055481, 0.9514222741127014, 0.9509723782539368]
zh.source3.LA400|gpt-3.5-turbo|TP2|avg_bert_score[P, R, F1] = 
[0.9507081508636475, 0.9503522515296936, 0.9504678249359131]
zh.source3.LA400|gpt-3.5-turbo|TP3|avg_bert_score[P, R, F1] = 
[0.9511091113090515, 0.9513288736343384, 0.9511555433273315]
zh.source3.LA400|NLLB|avg_bert_score[P, R, F1] = 
[0.9368485808372498, 0.9347572326660156, 0.9357397556304932]
zh.source3.LA400|Google|avg_bert_score[P, R, F1] = 
[0.9543501138687134, 0.9517157077789307, 0.9529737830162048]
zh.source3.LA400|DeepL|avg_bert_score[P, R, F1] = 
[0.945054

zh.source|NLLB|avg_bert_score[P, R, F1] = 
[0.931108295917511, 0.9291131496429443, 0.9300329685211182]
zh.source|Google|avg_bert_score[P, R, F1] = 
[0.9504942893981934, 0.9476957321166992, 0.949028730392456]
zh.source|DeepL|avg_bert_score[P, R, F1] = 
[0.9407556056976318, 0.9398877024650574, 0.9402612447738647]


## bleurt_score
- intall: https://github.com/google-research/bleurt

### bleurt_score sample

In [15]:
import tensorflow as tf
import os
from bleurt import score as bleurt_score
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
tf.get_logger().setLevel('ERROR')
checkpoint = "bleurt/test_checkpoint"

references = ["This is a test."]
candidates = ["This is the test."]

bleurt_scorer = bleurt_score.BleurtScorer()
bleurt_scores = bleurt_scorer.score(references=references, candidates=candidates)[0]
# assert isinstance(scores, list) and len(scores) == 1
print(bleurt_scores)

0.7463240027427673


### bleurt_score evaluation

In [16]:
list_src_name = ['zh.source1.CL_SA450', 'zh.source2.CT-SA350', 'zh.source3.LA400', 'zh.source']
list_ref_name = ['test.true.en1.CL_SA450', 'test.true.en2.CT-SA350', 'test.true.en3.LA400', 'test.true.en']
# GPT individual
list_tslt_folder = ['NLLB', 'Google', 'DeepL']

import tensorflow as tf
import os
from bleurt import score as bleurt_score
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
tf.get_logger().setLevel('ERROR')
checkpoint = "bleurt/test_checkpoint"
def read_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    return [line.strip() for line in lines]

bleurt_scorer = bleurt_score.BleurtScorer()

for i_name in range(len(list_src_name)):
    src_name = list_src_name[i_name]
    ref_name = list_ref_name[i_name]

    path_ref = 'dataset/'+ref_name
    for tpi in range(1,3+1):
        id_src = ""
        if i_name != 3:
            id_src = str(i_name+1)
        path_tslt = 'translated/gpt-3.5-turbo_API/zh.source'+id_src+'_tp'+str(tpi)+'_tslt_en.txt'

        refs = read_file(path_ref)
        tslts = read_file(path_tslt)

        bleurt_scores = [bleurt_scorer.score(references=[r], candidates=[t])[0] for r,t in zip(refs, tslts)]
        avg_bleurt_score = sum(bleurt_scores)/len(bleurt_scores)

        print("%s|gpt-3.5-turbo|TP%d|avg_bleurt_score = "%(src_name,tpi))
        print(avg_bleurt_score)

    for tslt_name in list_tslt_folder:
        path_tslt = 'translated/'+tslt_name+'_API/'+tslt_name+'_'+src_name+'_tslt_en.txt'

        refs = read_file(path_ref)
        tslts = read_file(path_tslt)

        bleurt_scores = [bleurt_scorer.score(references=[r], candidates=[t])[0] for r,t in zip(refs, tslts)]
        avg_bleurt_score = sum(bleurt_scores)/len(bleurt_scores)

        print("%s|%s|sentence_bleu_avg = "%(src_name,tslt_name))
        print(avg_bleurt_score)

zh.source1.CL_SA450|gpt-3.5-turbo|TP1|avg_bleurt_score = 
0.4034105663994948
zh.source1.CL_SA450|gpt-3.5-turbo|TP2|avg_bleurt_score = 
0.39062041670084
zh.source1.CL_SA450|gpt-3.5-turbo|TP3|avg_bleurt_score = 
0.40343021182550326
zh.source1.CL_SA450|NLLB|sentence_bleu_avg = 
0.24216023335854212
zh.source1.CL_SA450|Google|sentence_bleu_avg = 
0.40325494291053876
zh.source1.CL_SA450|DeepL|sentence_bleu_avg = 
0.3409347960518466
zh.source2.CT-SA350|gpt-3.5-turbo|TP1|avg_bleurt_score = 
0.41219661135758673
zh.source2.CT-SA350|gpt-3.5-turbo|TP2|avg_bleurt_score = 
0.4045568690129689
zh.source2.CT-SA350|gpt-3.5-turbo|TP3|avg_bleurt_score = 
0.415498162571873
zh.source2.CT-SA350|NLLB|sentence_bleu_avg = 
0.21882923743554525
zh.source2.CT-SA350|Google|sentence_bleu_avg = 
0.4421633297630719
zh.source2.CT-SA350|DeepL|sentence_bleu_avg = 
0.37189702380980766
zh.source3.LA400|gpt-3.5-turbo|TP1|avg_bleurt_score = 
0.29131466342136264
zh.source3.LA400|gpt-3.5-turbo|TP2|avg_bleurt_score = 
0.2923653

## Entity-Aware BERTScore + Commonsense Reasoning Accuracy
- https://github.com/YutongWang1216/CR-NMT

conda activate ev4nlpCr2
cd D:\BaidNetdiskSync_CXH\nlp2ct_cr_project\CR-NMT-main

python ent_bertscore/ent_score.py     --candidate data/ptnmt/cr-test.hyp     --r_reference data/cr_testset/test.true.en     --c_reference data/cr_testset/test.wrong.en     --weight 1.4     --outdir example/

python CR_accuracy/calculate_accuracy.py     --r_score example/true.score.bert_1.4     --c_score example/wrong.score.bert_1.4

### generate cmd code

In [67]:
commands = ['conda activate ev4nlpCr2',
'cd D:/BaidNetdiskSync_CXH/nlp2ct_cr_project/CR-NMT-main']

In [68]:
list_src_name = ['zh.source']
list_ref_name = ['test.true.en']
# GPT individual
list_tslt_folder = ['NLLB', 'Google', 'DeepL']

for i_name in range(len(list_src_name)):
    src_name = list_src_name[i_name]
    ref_name = list_ref_name[i_name]

    path_ref = '../dataset/'+ref_name
    path_candhyp = '../candidate_hyp/'+ref_name+'.hyp'
    for tpi in range(1,3+1):
        id_src = ""
        # if i_name != 3:
        #     id_src = str(i_name+1)
        path_tslt = '../translated/gpt-3.5-turbo_API/zh.source'+id_src+'_tp'+str(tpi)+'_tslt_en.txt'

        path_outdir = 'model/'+src_name+'_gpt-3.5-turbo_tp'+str(tpi)

        cmd1 = """python ent_bertscore/ent_score.py \
        --candidate data/ptnmt/cr-test.hyp \
        --r_reference %s \
        --c_reference %s \
        --weight 1.4 \
        --outdir %s/"""%(path_ref, path_tslt, path_outdir)
        cmd1=cmd1.replace('\n',' ')
        print("%s|gpt-3.5-turbo|TP%d|cmd1:"%(src_name,tpi))
        print(cmd1+'\n')
        commands.append(cmd1.strip())

        cmd2 = """python CR_accuracy/calculate_accuracy.py \
        --r_score %s/true.score.bert_1.4 \
        --c_score %s/wrong.score.bert_1.4"""%(path_outdir, path_outdir)
        cmd2+=' >> cmd_output.txt'
        print("%s|gpt-3.5-turbo|TP%d|cmd2:"%(src_name,tpi))
        print(cmd2+'\n\n-----\n')
        commands.append(cmd2.strip())

    for tslt_name in list_tslt_folder:
        path_tslt = '../translated/'+tslt_name+'_API/'+tslt_name+'_'+src_name+'_tslt_en.txt'
        path_outdir = 'model/'+src_name+'_'+tslt_name
    
        cmd1 = """python ent_bertscore/ent_score.py \
        --candidate data/ptnmt/cr-test.hyp \
        --r_reference %s \
        --c_reference %s \
        --weight 1.4 \
        --outdir %s/"""%(path_ref, path_tslt, path_outdir)
        cmd1=cmd1.replace('\n',' ')
        print("%s|%s|cmd1 = "%(src_name,tslt_name))
        print(cmd1+'\n')
        commands.append(cmd1.strip())

        cmd2 = """python CR_accuracy/calculate_accuracy.py \
        --r_score %s/true.score.bert_1.4 \
        --c_score %s/wrong.score.bert_1.4"""%(path_outdir, path_outdir)
        cmd2+=' >> cmd_output.txt'
        print("%s|%s|cmd2 = "%(src_name,tslt_name))
        print(cmd2+'\n\n-----\n')
        commands.append(cmd2.strip())

zh.source|gpt-3.5-turbo|TP1|cmd1:
python ent_bertscore/ent_score.py         --candidate data/ptnmt/cr-test.hyp         --r_reference ../dataset/test.true.en         --c_reference ../translated/gpt-3.5-turbo_API/zh.source_tp1_tslt_en.txt         --weight 1.4         --outdir model/zh.source_gpt-3.5-turbo_tp1/

zh.source|gpt-3.5-turbo|TP1|cmd2:
python CR_accuracy/calculate_accuracy.py         --r_score model/zh.source_gpt-3.5-turbo_tp1/true.score.bert_1.4         --c_score model/zh.source_gpt-3.5-turbo_tp1/wrong.score.bert_1.4 >> cmd_output.txt

-----

zh.source|gpt-3.5-turbo|TP2|cmd1:
python ent_bertscore/ent_score.py         --candidate data/ptnmt/cr-test.hyp         --r_reference ../dataset/test.true.en         --c_reference ../translated/gpt-3.5-turbo_API/zh.source_tp2_tslt_en.txt         --weight 1.4         --outdir model/zh.source_gpt-3.5-turbo_tp2/

zh.source|gpt-3.5-turbo|TP2|cmd2:
python CR_accuracy/calculate_accuracy.py         --r_score model/zh.source_gpt-3.5-turbo_tp2/true.

In [69]:
commands = '\n'.join(commands)+'\n'
commands

'conda activate ev4nlpCr2\ncd D:/BaidNetdiskSync_CXH/nlp2ct_cr_project/CR-NMT-main\npython ent_bertscore/ent_score.py         --candidate data/ptnmt/cr-test.hyp         --r_reference ../dataset/test.true.en         --c_reference ../translated/gpt-3.5-turbo_API/zh.source_tp1_tslt_en.txt         --weight 1.4         --outdir model/zh.source_gpt-3.5-turbo_tp1/\npython CR_accuracy/calculate_accuracy.py         --r_score model/zh.source_gpt-3.5-turbo_tp1/true.score.bert_1.4         --c_score model/zh.source_gpt-3.5-turbo_tp1/wrong.score.bert_1.4 >> cmd_output.txt\npython ent_bertscore/ent_score.py         --candidate data/ptnmt/cr-test.hyp         --r_reference ../dataset/test.true.en         --c_reference ../translated/gpt-3.5-turbo_API/zh.source_tp2_tslt_en.txt         --weight 1.4         --outdir model/zh.source_gpt-3.5-turbo_tp2/\npython CR_accuracy/calculate_accuracy.py         --r_score model/zh.source_gpt-3.5-turbo_tp2/true.score.bert_1.4         --c_score model/zh.source_gpt-3.5-tu

In [70]:
import subprocess

# 使用subprocess.Popen启动一个子进程
# 使用cmd.exe在Windows上，使用/bin/bash在Unix或Linux上
shell_process = subprocess.Popen("cmd.exe", stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# 将命令发送到子进程
stdout, stderr = shell_process.communicate(input=commands)

# 打印命令输出
print(stdout)

# 如果有错误，打印错误
if stderr:
    print(stderr)

Microsoft Windows [Version 10.0.22621.2861]
(c) Microsoft Corporation. All rights reserved.

(base) d:\BaidNetdiskSync_CXH\nlp2ct_cr_project>conda activate ev4nlpCr2

(ev4nlpCr2) d:\BaidNetdiskSync_CXH\nlp2ct_cr_project>cd D:/BaidNetdiskSync_CXH/nlp2ct_cr_project/CR-NMT-main

(ev4nlpCr2) D:\BaidNetdiskSync_CXH\nlp2ct_cr_project\CR-NMT-main>python ent_bertscore/ent_score.py         --candidate data/ptnmt/cr-test.hyp         --r_reference ../dataset/test.true.en         --c_reference ../translated/gpt-3.5-turbo_API/zh.source_tp1_tslt_en.txt         --weight 1.4         --outdir model/zh.source_gpt-3.5-turbo_tp1/
Current weight of commonsense entities: 1.40
calculating scores...
computing bert embedding.
computing greedy matching.
done in 3.86 seconds, 310.89 sentences/sec
calculating scores...
computing bert embedding.
computing greedy matching.
done in 3.42 seconds, 351.13 sentences/sec

(ev4nlpCr2) D:\BaidNetdiskSync_CXH\nlp2ct_cr_project\CR-NMT-main>python CR_accuracy/calculate_accura